In [183]:
import subprocess as sp
import gurobipy as gp
import sage.all as sg
from sage.all import libgap as lg
import time
import os
import copy
import cplex as cp
import math
import scipy

In [184]:
class orbital_partition:
    def __init__(self, mdl, orbits):
        num_col_orbits = 0
        num_row_orbits = 0
        self.num_col_orbits = 0
        self.num_row_orbits = 0
        self.orbit_col = []
        self.orbit_col_start = [0]
        self.orbit_row = []
        self.orbit_row_start = [0]
        self.col_orbit = {}
        self.row_orbit = {}
        self.col_orbit_rep = {}
        self.row_orbit_rep = {}
        self.col_orbit_size = {}
        self.row_orbit_size = {}
        for orbit in orbits:
            if orbit[0] < mdl.variables.get_num():
                for el in orbit:
                    self.orbit_col.append(el)
                    self.col_orbit[int(el)] = num_col_orbits
                    if self.col_orbit_size.get(num_col_orbits) == None:
                        self.col_orbit_size[num_col_orbits] = 0
                    self.col_orbit_size[num_col_orbits] += 1
                self.orbit_col_start.append(len(self.orbit_col))
                self.col_orbit_rep[num_col_orbits] = int(el)
                num_col_orbits += 1
                self.num_col_orbits += 1
            else:
                for el in orbit:
                    self.orbit_row.append(el - mdl.variables.get_num())
                    self.row_orbit[int(el - mdl.variables.get_num())] = num_row_orbits
                    if self.row_orbit_size.get(num_row_orbits) == None:
                        self.row_orbit_size[num_row_orbits] = 0
                    self.row_orbit_size[num_row_orbits] += 1
                self.orbit_row_start.append(len(self.orbit_row))
                self.row_orbit_rep[num_row_orbits] = int(el - mdl.variables.get_num())
                num_row_orbits += 1
                self.num_row_orbits += 1
    def get_num_orbits(self):
        return self.num_col_orbits, self.num_row_orbits
    def get_orbits(self):
        return self.col_orbit, self.row_orbit
    def get_orbit_reps(self):
        return self.col_orbit_rep, self.row_orbit_rep
    def get_orbit_sizes(self):
        return self.col_orbit_size, self.row_orbit_size

class orbital_cut_generation_stats:
    def __init__(self):
        self.lift_iter = 0
        self.lift_cuts = 0

In [222]:
def cplex_read_model(model_file):
    return cp.Cplex(model_file)

def aggregate_A_mat(mdl, orbits):
    agg_mdl = cp.Cplex()
    num_col_orbits, num_row_orbits = orbits.get_num_orbits()
    col_orbit, row_orbit = orbits.get_orbits()
    col_orbit_rep, row_orbit_rep = orbits.get_orbit_reps()
    col_orbit_size, row_orbit_size = orbits.get_orbit_sizes()
    # ADD AGGREGATE ROWS
    rhs = mdl.linear_constraints.get_rhs()
    names = mdl.linear_constraints.get_names()
    sense = mdl.linear_constraints.get_senses()
    agg_rhs = []
    agg_name = []
    agg_sense = []
    for agg_row in range(num_row_orbits):
        rep = row_orbit_rep.get(agg_row)
        agg_rhs.append(rhs[rep] * row_orbit_size[agg_row])
        agg_sense.append(sense[rep])
        agg_name.append(names[rep])
    agg_mdl.linear_constraints.add(rhs = agg_rhs, senses = agg_sense, names = agg_name)
    # BUILD AGGREGATE COLUMNS
    objs = mdl.objective.get_linear()
    lbs = mdl.variables.get_lower_bounds()
    ubs = mdl.variables.get_upper_bounds()
#     types = mdl.variables.get_types()
    names = mdl.variables.get_names()
    agg_obj = []
    agg_col_vec = []
    agg_lb = []
    agg_ub = []
    agg_type = []
    agg_name = []
    for agg_col in range(num_col_orbits):
        agg_value = {}
        rep = col_orbit_rep.get(agg_col)
        index, value = mdl.variables.get_cols(rep).unpack()
        for ind, val in zip(index, value):
            row_orb = row_orbit.get(ind)
            if agg_value.get(row_orb) == None:
                agg_value[row_orb] = 0
            agg_value[row_orb] += val
        row_val = []
        row_idx = []
        for agg_row, val in agg_value.items():
            row_val.append(val)
            row_idx.append(int(agg_row))
        agg_obj.append(objs[rep])
        agg_col_vec.append(cp.SparsePair(row_idx, row_val))
        agg_lb.append(lbs[rep] * col_orbit_size[agg_col])
        agg_ub.append(ubs[rep] * col_orbit_size[agg_col])
        agg_type.append("C")
        agg_name.append(names[rep])
    agg_mdl.variables.add(obj = agg_obj, lb = agg_lb, ub = agg_ub, types = agg_type,
                          names = agg_name, columns = agg_col_vec)
#     agg_mdl.write("agg_model_cplex.lp")
    return agg_mdl

def solve_cplex(mdl):
#     agg_mdl.parameters.lpmethod.set(agg_mdl.parameters.lpmethod.values.primal)
    mdl.set_problem_type(agg_mdl.problem_type.LP)
    mdl.solve()
    
def generate_cuts(full_mdl, mdl, group, first_group, orbits):
    return_orbits = []
    return_hash_table = {}
    return_node_hash = {}
    num_col = mdl.variables.get_num()
    num_row = mdl.linear_constraints.get_num()
    num_tot = num_col + num_row
    fract_rows = []
    fract_rhs = []
    row_base_idx, row_base_val = mdl.solution.basis.get_header()
    for i_row, base_idx in enumerate(row_base_idx):
        row_val = row_base_val[i_row]
        if (abs(round(row_val) - row_val) > 1e-6):
#             and i_row > -1):
            fract_rows.append(i_row)
        fract_rhs.append(row_val)
    # GENERATE GOMORY CUTS FOR ALL FRACTIONAL ROWS
    binvarow = mdl.solution.advanced.binvarow()
    binvrow = mdl.solution.advanced.binvrow()
    for i_row, val in enumerate(binvarow):
        print(binvarow[i_row] + binvrow[i_row], "|", row_base_val[i_row])
    for i_row in fract_rows:
        full_num_col = full_mdl.variables.get_num()
        full_num_row = full_mdl.linear_constraints.get_num()
        full_num_tot = full_num_col + full_num_row
        binvar = binvarow[i_row]
        binvr = binvrow[i_row]
        reduced_row = binvar + binvr
        for i_slack in range(num_col, num_tot):
            sense = mdl.linear_constraints.get_senses(i_slack - num_col)
            if sense == "G":
                reduced_row[i_slack] *= -1
        for i_col in range(num_tot):
            reduced_row[i_col] -= math.floor(reduced_row[i_col])
        fract_rhs[i_row] -= math.floor(fract_rhs[i_row])
        for i_slack in range(num_col, num_tot):
            value = {}
            if (abs(round(reduced_row[i_slack]) - reduced_row[i_slack])) < 1e-6:
                continue
            sense = mdl.linear_constraints.get_senses(i_slack - num_col)
            if sense == "G":
                slack_coeff = -1
            else:
                slack_coeff = 1
            a_idx, a_val = mdl.linear_constraints.get_rows(i_slack - num_col).unpack()
            a_rhs = mdl.linear_constraints.get_rhs(i_slack - num_col)
            for idx, val in zip(a_idx, a_val):
                reduced_row[idx] += reduced_row[i_slack] * slack_coeff * -val
            fract_rhs[i_row] += -1 * reduced_row[i_slack] * slack_coeff * a_rhs
            reduced_row[i_slack] = 0
        reduced_row = [sg.RealNumber(val) for val in reduced_row[:num_col]]
        # GENERATE GOMORY CUT ORBITS AT AGGREGATE AND EXTENDED LEVELS
        (agg_cut_csr, full_cut_csr, new_orbits, new_orbit_sizes, 
         cut_hash_table, cut_node_hash) = lift_cut(reduced_row, group, first_group,
                                                   orbits, full_num_tot)
        return_orbits += new_orbits
        return_hash_table.update(cut_hash_table)
        return_node_hash.update(cut_node_hash)
        cut_rows = []
        cut_rows_sense = []
        cut_rows_rhs = []
        cut_rows_name = []
        num_agg_cut = agg_cut_csr.shape[0]
        indptr = agg_cut_csr.indptr
        indices = agg_cut_csr.indices
        data = agg_cut_csr.data
        for cut_row in range(num_agg_cut):
            cut_val = []
            cut_idx = []
            for i_cut in range(indptr[cut_row], indptr[cut_row + 1]):
                cut_val.append(float(data[i_cut]))
                cut_idx.append(int(indices[i_cut]))
            cut_rows.append(cp.SparsePair(cut_idx, cut_val))
            cut_rows_sense.append("G")
            cut_rows_rhs.append(fract_rhs[i_row] * new_orbit_sizes.get(cut_row))
#             cut_rows_name.append("cut")
        mdl.linear_constraints.add(lin_expr = cut_rows, senses = cut_rows_sense,
                                   rhs = cut_rows_rhs, names = cut_rows_name)
        cut_rows = []
        cut_rows_sense = []
        cut_rows_rhs = []
        cut_rows_name = []
        num_full_cut = full_cut_csr.shape[0]
        indptr = full_cut_csr.indptr
        indices = full_cut_csr.indices
        data = full_cut_csr.data
        for cut_row in range(num_full_cut):
            cut_val = []
            cut_idx = []
            for i_cut in range(indptr[cut_row], indptr[cut_row + 1]):
                cut_val.append(float(data[i_cut]))
                cut_idx.append(int(indices[i_cut]))
            cut_rows.append(cp.SparsePair(cut_idx, cut_val))
            cut_rows_sense.append("G")
            cut_rows_rhs.append(fract_rhs[i_row])
        full_mdl.linear_constraints.add(lin_expr = cut_rows, senses = cut_rows_sense,
                                       rhs = cut_rows_rhs, names = cut_rows_name)
#     mdl.write("agg_model_cplex_after_cuts.lp")
    mdl.write("cut_added_agg_model_cplex.lp")
    full_mdl.write("cut_added_model_cplex.lp")
    full_mdl.write("cut_added_model_cplex.mps")
    return return_orbits, return_hash_table, return_node_hash

def lift_cut(cut, group, first_group, orbits, num_tot):
    num_full_cut = copy.deepcopy(num_tot)
    num_full_row = copy.deepcopy(num_tot)
    num_row_orbits = 0
    new_cut_orbit_size = {}
    new_cut_orbit ={}
    node_orbit = {}
    lifted_cut = [0 for col in orbits.col_orbit.keys()]
    agg_cut_val = []
    agg_cut_idx = []
    agg_cut_start = [0]
    full_cut_val = []
    full_cut_idx = []
    full_cut_start = [0]
    # GENERATE ALL CUTS IN THE ORBIT OF ORIGINAL CUT USING ORIGINAL GROUP G
    for col, coeff in enumerate(cut):
        for idx in range(orbits.orbit_col_start[col], orbits.orbit_col_start[col + 1]):
            var = orbits.orbit_col[idx]
            lifted_cut[var] = coeff
    cut_orbit = lg.Orbit(lg(group), lifted_cut, lg.Permuted)
    ##### Problem 
    new_orbits = []
    cut_hash_table = {}
    cut_node_hash = {}
    # LIFT THE CUTS TO THE ORIGINAL MODEL
    for cut_orb in cut_orbit:
        cut_key = ",".join(str(el) for el in cut_orb)
        node_key = cut_hash_table.get(cut_key)
        for col, coeff in enumerate(cut_orb):
            if (abs(coeff.sage() - 0) < 1e-6):
                continue
            full_cut_val.append(float(coeff))
            full_cut_idx.append(int(col))
        full_cut_start.append(len(full_cut_val))
        if node_orbit.get(node_key) != None:
            continue
        stab_orb = lg.Orbit(lg(group), cut_orb, lg.Permuted)
        new_orbit = []
        for orb in stab_orb:
            orb_key = ",".join(str(el) for el in orb)
            cut_hash_table[orb_key] = num_full_cut
            cut_node_hash[num_full_cut] = orb_key
            new_orbit.append(num_full_cut)
            node_orbit[num_full_cut] = num_row_orbits
            num_full_cut += 1
        new_cut_orbit_size[num_row_orbits] = len(new_orbit)
        num_row_orbits += 1
        new_orbits.append(new_orbit)
    full_cut_csr = scipy.sparse.csr_matrix((full_cut_val, full_cut_idx, full_cut_start))
    # GENERATE AGGREGATE CUTS FOR THE NEW ORBITS IN THE AGGREGATE SPACE
    for agg_col in range(orbits.num_col_orbits):
        agg_coeff = {}
        rep = orbits.col_orbit_rep.get(agg_col)
        for full_cut in range(num_full_row, num_full_cut):
#         for full_cut in range(num_full_row, num_full_row + 1):
            agg_cut = node_orbit.get(full_cut)
            cut_str = cut_node_hash.get(full_cut)
            cut_str = cut_str.split(",")
            cut = [sg.RealNumber(el) for el in cut_str]
            if agg_coeff.get(agg_cut) == None:
                agg_coeff[agg_cut] = 0
            agg_coeff[agg_cut] += (cut[rep]) 
        for cut_idx, coeff in agg_coeff.items():
            if (abs(coeff - 0) < 1e-6):
                continue
            agg_cut_val.append(float(coeff))
            agg_cut_idx.append(int(cut_idx))
        agg_cut_start.append(len(agg_cut_val))
    agg_cut_csr = scipy.sparse.csc_matrix((agg_cut_val, agg_cut_idx, agg_cut_start)).tocsr()
    return (agg_cut_csr, full_cut_csr, new_orbits, new_cut_orbit_size, cut_hash_table, cut_node_hash)
        
def write_orbits(orbits, num_col, num_row):
    for orb in orbits:
        orb.sort()
    _num_col = 0
    _num_row = 0
    for orbit in orbits:
        if orbit[0] <= num_col:
            _num_col += 1
        else:
            _num_row += 1
    with open("./sage_orbits.txt", "w") as f:
        for orbit in orbits[ : _num_col]:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
        for orbit in orbits[_num_col : _num_col + _num_row]:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
    print(orbits)
    print("Orbits Written")
    return _num_col, _num_row

def read_highs_basis(basis_file, col_basis, row_basis, 
                     agg_num_col, agg_num_row):
    col_basis.clear()
    row_basis.clear()
    with open(basis_file) as f:
        lines = f.readlines()
        for num, line in enumerate(lines):
            if num == 3:
                for col, status in enumerate(line.split(" ")[:-1]):
                    col_basis[col] = int(status)
            if num == 5:
                for row, status in enumerate(line.split(" ")[:-1]):
                    row_basis[row] = int(status)
                    
def lift_highs_basis(prev_col_basis, prev_row_basis, col_basis, row_basis, 
                     agg_col_rep, col_agg, agg_row_rep, row_agg, prev_agg_col_rep,
                     prev_col_agg, prev_agg_row_rep, prev_row_agg, agg_num_col,
                     agg_num_row, parent):
    # Set column basis
    num_linkers = len(parent)
    col_basis.clear()
    num_basic = 0
    for col, rep in agg_col_rep.items():
        old_col = prev_col_agg.get(rep)
        prev_stat = prev_col_basis.get(old_col)
        col_basis[col] = prev_stat
        if (prev_stat):
            num_basic += 1
    for link in range(num_linkers):
        col_basis[link + agg_num_col] = 0
    # Set row basis
    row_basis.clear()
    for row in agg_row_rep.keys():
        row_basis[row] = 1
#     print(prev_agg_row_rep.items())
    for old_row, rep in prev_agg_row_rep.items():
        prev_stat = prev_row_basis.get(old_row)
        row = row_agg.get(rep)
        row_basis[row] = prev_stat
#         print("old_row: %d" % old_row)
#         print("old_rep: %d" % (rep - 1))
#         print("new_row: %d" % row)
#         print("basis_stat: %d" % prev_stat)
#         input()
    for link in range(num_linkers):
        row_basis[link + agg_num_row] = 0
    for item in row_basis.values():
        if item == 1:
            num_basic += 1
    print(len(row_basis))
    print(num_basic)
    print(col_basis)
    print(row_basis)
    input()
        
def write_highs_basis(basis_file, col_basis, row_basis):
    with open(basis_file, "w") as f:
        f.write("HiGHS v1\nValid\n")
        num_col = len(col_basis)
        num_row = len(row_basis)
        f.write("# Columns %d\n" % num_col)
        for col, status in col_basis.items():
            if col == num_col - 1:
                f.write(str(status) + "\n")
                break
            f.write(str(status) + " ")
        f.write("# Rows %d\n" % num_row)
        for row, status in row_basis.items():
            if row == num_row - 1:
                f.write(str(status) + "\n")
                break
            f.write(str(status) + " ")

def write_initial_highs_basis(basis_file, agg_num_col, agg_num_row):
    with open(basis_file, "w") as f:
        f.write("HiGHS v1\nValid\n")
        f.write("# Columns %d\n" % agg_num_col)
        for col in range(agg_num_col):
            if col == agg_num_col - 1:
                f.write(str(0) + "\n")
                break
            f.write(str(0) + " ")
        f.write("# Rows %d\n" % agg_num_row)
        for row in range(agg_num_row):
            if row == agg_num_row - 1:
                f.write(str(1) + "\n")
                break
            f.write(str(1) + " ")
            
def get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                              col_agg, prev_col_agg,
                              agg_col_rep, prev_agg_col_rep,
                              prev_col_basis):
    parent = []
    child = []
    if not prev_agg_num_col:
        return parent, child
    for orb, rep in agg_col_rep.items():
        prev_orb = prev_col_agg.get(rep)
        prev_orb_rep = prev_agg_col_rep.get(prev_orb)
        if orb == prev_orb:
            continue
        if rep == prev_orb_rep: 
            continue
        if prev_col_basis.get(prev_orb) != 1:
            continue
        parent_orb = col_agg.get(prev_orb_rep)
        parent.append(parent_orb)
        child.append(orb)
    return parent, child

def write_orbit_linkers(parent, child):
    with open("./orbit_linkers.txt", "w") as f:
        f.write("Parents %d\n" % len(parent))
        for el in parent:
            f.write(str(el) + " ")
        f.write("\n")
        f.write("Children %d\n" % len(child))
        for el in child:
            f.write(str(el) + " ")
        f.write("\n")
        
def pair_orbit_with_aggregate_column(orbits, num_agg_col):
    agg_col_rep = {}
    col_agg = {}
    for key, orbit in enumerate(orbits[ : num_agg_col]):
        agg_col_rep[key] = orbit[0]
        for el in orbit:
            col_agg[el] = key
    return agg_col_rep, col_agg

def pair_orbit_with_aggregate_row(orbits, num_agg_col, num_agg_row):
    agg_row_rep = {}
    row_agg = {}
    for key, orbit in enumerate(orbits[num_agg_col : num_agg_col + num_agg_row]):
        agg_row_rep[key] = orbit[0]
        for el in orbit:
            row_agg[el] = key
    return agg_row_rep, row_agg
                        
def get_stabilizer_group_orbits(group):
    group_orbits = group.orbits()
    for orb in group_orbits:
        if len(orb) > 1:
            stable_pnt = orb[0]
            stabilizer_group = group.stabilizer(stable_pnt)
            print("New Orbits Calculated")
            return stabilizer_group, stabilizer_group.orbits()
    print("Discrete Orbits Calculated")
    return group, group.orbits()
        
def concatenate_orbits(real_orbits, cut_orbits):
    out = []
    for orb in real_orbits:
        out.append(orb)
    for orb in cut_orbits:
        out.append(orb)
    print("Orbits Concatenated")
    return out

def call_coin_cgl(lp_path):
    cgl_cut_file = "./CutGeneration/cut_txt_files/cuts.txt"
    cgl_out = sp.Popen(["./CutGeneration/generateMIPCuts",lp_path,"./sage_orbits.txt"], shell = False, 
                       stdout = sp.DEVNULL)
    cgl_return, cgl_err = cgl_out.communicate()
    print("Cgl Solved Aggregate LP")
    return cgl_return, cgl_err

def split_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group):
    new_cut_orbits = []
    nodes_split = {}
    for cut_orbit in cut_orbits:
        if len(cut_orbit) == 1: 
            new_cut_orbits.append(cut_orbit)
            continue
        for node in cut_orbit:
            if nodes_split.get(node) != None:
                continue
            cut_str = cut_vectors.get(node)
            cut_str = cut_str.split(",")
            cut_vec = [sg.RealNumber(el) for el in cut_str]
            orbs = lg.Orbit(lg(stab_group), cut_vec, lg.Permuted)
            add = list()
            for orb in orbs:
                cut_key = ",".join(str(el) for el in orb)
                cut_node = cut_vectors_node.get(cut_key)
                if cut_node is not None:
                    add.append(cut_node)
                    nodes_split[cut_node] = 1
            new_cut_orbits.append(add)
    return new_cut_orbits
            
    
def generate_cut_orbits(sage_perm_group, lp_path, cut_orbits, orig_num_row,
                        prev_agg_row_rep, prev_row_agg, cut_vectors, cut_vectors_node):
    dup_cut_set = set()
    cut_rhs = []
    cut_added_model = gp.read(lp_path)
#     orig_cols = orig_model.getVars()
    cols = cut_added_model.getVars()
    rows = cut_added_model.getConstrs()
    num_nodes = len(cols) + len(rows) + 1
    num_cut = len(prev_agg_row_rep)
    num_total_agg_row = len(prev_agg_row_rep)
    with open("./cuts.txt") as cuts_f:
        lines = cuts_f.readlines()
        for line in lines:
            new_orbits = []
#             print("num_cut %d" % num_cut)
#             print("num_node %d" % num_nodes)
            prev_agg_row_rep[num_cut] = num_nodes
            prev_row_agg[num_nodes] = num_cut
            splt = line.split(",")
            cut = []
            rhs = (sg.RealNumber(splt[-1][:-1]))
            sense = splt[-2]
            for coeff in splt[:-2]:
                cut.append(sg.RealNumber(coeff))
            if sense == "L":
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) <= rhs
            elif sense == "G":
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) >= rhs
            else:
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) == rhs
            cut_added_model.addConstr(expr)
            new_orbits.append(num_nodes)
            cut_vectors[num_nodes] = copy.deepcopy(cut)
            cut_vectors_node[tuple(copy.deepcopy(cut))] = num_nodes
            num_nodes += 1
#             num_cut += 1
            dup_cut_set.add(tuple(cut))
#             #### Only uses partial many orbital cuts as opposed to full orbital cuts
#             cut.extend([0 for i in range(orig_num_row)])
#             for gen in sage_perm_group.gens():
#                 perm = sg.Permutation(gen.cycle_tuples(singletons=True))
#                 new_cut = perm.action(cut)
#                 new_cut = new_cut[:len(cols)]
#                 if tuple(new_cut) in dup_cut_set:
#                     continue
#                 print(new_cut)
#                 dup_cut_set.add(tuple(new_cut))
#                 if sense == "L":
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) <= rhs
#                 elif sense == "G":
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) >= rhs
#                 else:
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) == rhs
#                 cut_added_model.addConstr(expr)
#                 new_orbits.append(num_nodes)
#                 prev_row_agg[num_nodes] = num_cut
#                 cut_vectors[num_nodes] = copy.deepcopy(new_cut)
#                 cut_vectors_node[tuple(copy.deepcopy(new_cut))] = num_nodes
#                 num_nodes += 1
#             num_cut += 1
              # Uses all orbital cuts that can be generated
#             new_orbits = []
#             cut.extend()
            symmetric_cuts = lg.Orbit(lg(sage_perm_group), cut, lg.Permuted)
            num_sym_cuts = len(symmetric_cuts)
#             if num_sym_cuts > 5:
#                 num_sym_cuts = 5
            for j in range(num_sym_cuts):
                sym_cut = symmetric_cuts[j]
                sym_cut = [sg.RealNumber(val) for val in sym_cut]
                if tuple(sym_cut) in dup_cut_set:
                    continue
                dup_cut_set.add(tuple(sym_cut))
                if sense == "L":
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) <= rhs
                elif sense == "G":
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) >= rhs
                else:
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) == rhs
                cut_added_model.addConstr(expr)
                new_orbits.append(num_nodes)
                prev_row_agg[num_nodes] = num_cut
                cut_vectors[num_nodes] = copy.deepcopy(sym_cut)
                cut_vectors_node[tuple(copy.deepcopy(sym_cut))] = num_nodes
                num_nodes += 1
            num_cut += 1
            print(new_orbits)
            input()
            cut_orbits.append(new_orbits)
    cut_added_model.write("highs_input_lp.mps")
#     orig_model.write("test.lp")
    print("Cut Orbits Generated")

# Fucntion to call everything in a loop on LPs in a file
def orbital_cut_generation(symm_lp):
    # Start time
    start_time = time.perf_counter()
    # Read initial mps file and get relevant lp graph info for saucy
    # Read initial lp in to gurobi to grab some descriptors
    orig_model, col, col_name, row, row_name = read_initial_mps(symm_lp)
    num_col, num_row = len(col), len(row)
    # initialize storage
    old_orbits = []
    orbits_to_write = []
    cut_orbits = []
    cut_vectors = {}
    cut_vectors_node = {}
    real_orbits = []
    prev_agg_col_rep = {}
    prev_agg_row_rep = {}
    prev_col_agg = {}
    prev_row_agg = {}
    prev_agg_num_col = 0
    prev_agg_num_row = 0
    # Call saucy on the lp graph
    saucy_out_file = open("./saucy_gens.txt", "w")
    saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                                stderr = saucy_out_file, shell = False)
    saucy_return, saucy_err = saucy_out.communicate()
    # Read in the orbit generators from saucy
    sage_gens = read_saucy_gens("./saucy_gens.txt", col, row)
    # Do sage permutation group from the generators
    sage_perm_group = PermutationGroup(sage_gens)
#     gap_group = lg(sage_perm_group) # Only needed if we want the full cut generated orbit
    # Get the initial set of orbits
    real_orbits = sage_perm_group.orbits()
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    # Write the initial orbits and all other pieces needed to solve ALP in highs
    agg_num_col, agg_num_row = write_orbits(orbits_to_write, num_col, num_row)
    prev_col_basis = {}
    col_basis = {}
    prev_row_basis = {}
    row_basis = {}
    agg_col_rep, col_agg = pair_orbit_with_aggregate_column(orbits_to_write, agg_num_col)
    agg_row_rep, row_agg = pair_orbit_with_aggregate_row(orbits_to_write, agg_num_col, agg_num_row)
    parent, child = get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                                              col_agg, prev_col_agg,
                                              agg_col_rep, prev_agg_col_rep,
                                              prev_col_basis)
    write_initial_highs_basis("./highs_input_basis.txt", agg_num_col, agg_num_row)
    write_orbit_linkers(parent, child)
    highs_out = sp.Popen(["../EQLPSolver/HiGHS-1-2-1/debugBuild/bin/highs",symm_lp,
                         "./sage_orbits.txt","./highs_input_basis.txt","./orbit_linkers.txt",
                         "--solver=orbital_cut_generation"])
    highs_out.wait()
    read_highs_basis("./python_input_basis.txt", prev_col_basis, prev_row_basis,
                     agg_num_col, agg_num_row)
    prev_agg_col_rep = copy.deepcopy(agg_col_rep)
    prev_agg_row_rep = copy.deepcopy(agg_row_rep)
    prev_col_agg = copy.deepcopy(col_agg)
    prev_row_agg = copy.deepcopy(row_agg)
    prev_agg_num_col = copy.deepcopy(agg_num_col)
    prev_agg_num_row = copy.deepcopy(agg_num_row)
    ########## Test Phase #################
    # Generate orbit of cutting plane in original space
    generate_cut_orbits(sage_perm_group, symm_lp, cut_orbits, num_row,
                        prev_agg_row_rep, prev_row_agg, cut_vectors, cut_vectors_node)
    # Get stabilizer group orbits and run again
    stab_group, real_orbits = get_stabilizer_group_orbits(sage_perm_group)
    cut_orbits = split_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group)
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    agg_num_col, agg_num_row = write_orbits(orbits_to_write, num_col, num_row)
    agg_col_rep, col_agg = pair_orbit_with_aggregate_column(orbits_to_write, agg_num_col)
    agg_row_rep, row_agg = pair_orbit_with_aggregate_row(orbits_to_write, agg_num_col, agg_num_row)
    parent, child = get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                                              col_agg, prev_col_agg,
                                              agg_col_rep, prev_agg_col_rep,
                                              prev_col_basis)
    lift_highs_basis(prev_col_basis, prev_row_basis, col_basis, row_basis, 
                     agg_col_rep, col_agg, agg_row_rep, row_agg, prev_agg_col_rep,
                     prev_col_agg, prev_agg_row_rep, prev_row_agg, agg_num_col,
                     agg_num_row, parent)
    write_highs_basis("./highs_input_basis.txt", col_basis, row_basis)
    write_orbit_linkers(parent, child)
    highs_out = sp.Popen(["../EQLPSolver/HiGHS-1-2-1/debugBuild/bin/highs","./highs_input_lp.mps",
                         "./sage_orbits.txt","./highs_input_basis.txt","./orbit_linkers.txt",
                         "--solver=orbital_cut_generation"])
    highs_out.wait()
    read_highs_basis("./python_input_basis.txt", prev_col_basis, prev_row_basis,
                     agg_num_col, agg_num_row)
    prev_agg_col_rep = copy.deepcopy(agg_col_rep)
    prev_agg_row_rep = copy.deepcopy(agg_row_rep)
    prev_col_agg = copy.deepcopy(col_agg)
    prev_row_agg = copy.deepcopy(row_agg)
    prev_agg_num_col = copy.deepcopy(agg_num_col)
    prev_agg_num_row = copy.deepcopy(agg_num_row)
    ########## Test Phase #################
    # Generate orbit of cutting plane in original space
    generate_cut_orbits(sage_perm_group, "./highs_input_lp.mps", cut_orbits, num_row,
                        prev_agg_row_rep, prev_row_agg, cut_vectors, cut_vectors_node)
    # Get stabilizer group orbits and run again
    stab_group, real_orbits = get_stabilizer_group_orbits(stab_group)
    cut_orbits = split_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group)
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    agg_num_col, agg_num_row = write_orbits(orbits_to_write, num_col, num_row)
    agg_col_rep, col_agg = pair_orbit_with_aggregate_column(orbits_to_write, agg_num_col)
    agg_row_rep, row_agg = pair_orbit_with_aggregate_row(orbits_to_write, agg_num_col, agg_num_row)
    parent, child = get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                                              col_agg, prev_col_agg,
                                              agg_col_rep, prev_agg_col_rep,
                                              prev_col_basis)
    lift_highs_basis(prev_col_basis, prev_row_basis, col_basis, row_basis, 
                     agg_col_rep, col_agg, agg_row_rep, row_agg, prev_agg_col_rep,
                     prev_col_agg, prev_agg_row_rep, prev_row_agg, agg_num_col,
                     agg_num_row, parent)
    write_highs_basis("./highs_input_basis.txt", col_basis, row_basis)
    write_orbit_linkers(parent, child)
    highs_out = sp.Popen(["../EQLPSolver/HiGHS-1-2-1/debugBuild/bin/highs","./highs_input_lp.mps",
                         "./sage_orbits.txt","./highs_input_basis.txt","./orbit_linkers.txt",
                         "--solver=orbital_cut_generation"])
    highs_out.wait()
    read_highs_basis("./python_input_basis.txt", prev_col_basis, prev_row_basis,
                     agg_num_col, agg_num_row)
    prev_agg_col_rep = copy.deepcopy(agg_col_rep)
    prev_agg_row_rep = copy.deepcopy(agg_row_rep)
    prev_col_agg = copy.deepcopy(col_agg)
    prev_row_agg = copy.deepcopy(row_agg)
    prev_agg_num_col = copy.deepcopy(agg_num_col)
    prev_agg_num_row = copy.deepcopy(agg_num_row)
    ########## Test Phase #################
    # Generate orbit of cutting plane in original space
    generate_cut_orbits(sage_perm_group, "./highs_input_lp.mps", cut_orbits, num_row,
                        prev_agg_row_rep, prev_row_agg, cut_vectors, cut_vectors_node)
    # Get stabilizer group orbits and run again
    stab_group, real_orbits = get_stabilizer_group_orbits(stab_group)
    cut_orbits = split_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group)
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    input()
    agg_num_col, agg_num_row = write_orbits(orbits_to_write, num_col, num_row)
    agg_col_rep, col_agg = pair_orbit_with_aggregate_column(orbits_to_write, agg_num_col)
    agg_row_rep, row_agg = pair_orbit_with_aggregate_row(orbits_to_write, agg_num_col, agg_num_row)
    parent, child = get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                                              col_agg, prev_col_agg,
                                              agg_col_rep, prev_agg_col_rep, 
                                              prev_col_basis)
#     print(cut_orbits)
#     print(agg_row_rep)
    lift_highs_basis(prev_col_basis, prev_row_basis, col_basis, row_basis, 
                     agg_col_rep, col_agg, agg_row_rep, row_agg, prev_agg_col_rep,
                     prev_col_agg, prev_agg_row_rep, prev_row_agg, agg_num_col,
                     agg_num_row, parent)
    write_highs_basis("./highs_input_basis.txt", col_basis, row_basis)
    input()
    write_orbit_linkers(parent, child)
    highs_out = sp.Popen(["../EQLPSolver/HiGHS-1-2-1/debugBuild/bin/highs","./highs_input_lp.mps",
                         "./sage_orbits.txt","./highs_input_basis.txt","./orbit_linkers.txt",
                         "--solver=orbital_cut_generation"])
    highs_out.wait()
    read_highs_basis("./python_input_basis.txt", prev_col_basis, prev_row_basis,
                     agg_num_col, agg_num_row)

def write_all_orbits(symm_lp):
    # Read initial lp in to gurobi to grab some descriptors
    col, col_name, row, row_name = read_initial_mps(symm_lp)
    # Call saucy on the lp graph
    saucy_out_file = open("./saucy_gens.txt", "w")
    saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                                stderr = saucy_out_file, shell = False)
    saucy_return, saucy_err = saucy_out.communicate()
    sage_gens = read_saucy_gens("./saucy_gens.txt", col, row)
    # Write all the orbits of stabilizer groups to a file until the orbits are discrete
    cut_orbits = []
    sage_perm_group = PermutationGroup(sage_gens)
    real_orbits = sage_perm_group.orbits()
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    write_orbits(orbits_to_write)
    stab_group, real_orbits = get_stabilizer_group_orbits(sage_perm_group)
    print(real_orbits)
    
def read_saucy_gens(saucy_file, col_idx, row_idx):
    sage_gens = []
    gen_idx = 0
    # Read in the orbit generators from saucy
    with open(saucy_file) as f:
        lines = f.readlines()
        for line in lines:
            if line[0] == "C":
                continue
            else:
                sage_gens.append([])
                orbits = re.findall(r"\((.*?)\)",line)
                for orb in orbits:
                    temp = []
                    for node in orb.split(" "):
                        if node in col_idx.keys():
                            temp.append(col_idx.get(node))
                        else:
                            temp.append(row_idx.get(node))
                    temp = tuple(temp)
                    sage_gens[gen_idx].append(temp)
                gen_idx += 1
    return sage_gens

def read_initial_mps(symm_lp):
    # Read in mps file and create dicitonary for variable names and indices 
    orig_model = gp.read(symm_lp)
    orig_cols = orig_model.getVars()
    orig_rows = orig_model.getConstrs()
    col_idx = {col.varName : i for i, col in enumerate(orig_cols)}
    col_name = {i : col.varName for i, col in enumerate(orig_cols)}
    row_idx = {row.constrName : i + len(orig_cols) for i, row in enumerate(orig_rows)}
    row_name = {i + len(orig_cols) : row.constrName for i, row in enumerate(orig_rows)}
    return orig_model, col_idx, col_name, row_idx, row_name

In [223]:
symm_lp = "../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps";
cut_add_lp = "./cut_added_model_cplex.mps"
# write_all_orbits(symm_lp)
# orbital_cut_generation(symm_lp)
# READ IN THE ORIGINAL MODEL FILE WITH CPLEX
o_mdl = cplex_read_model(symm_lp)
o_mdl_num_col = o_mdl.variables.get_num()
o_mdl_num_row = o_mdl.linear_constraints.get_num()
o_mdl_col_names = o_mdl.variables.get_names()
o_mdl_cols = {name : idx for idx, name in enumerate(o_mdl_col_names)}
o_mdl_row_names = o_mdl.linear_constraints.get_names()
o_mdl_rows = {name : idx + o_mdl_num_col for idx, name in enumerate(o_mdl_row_names)}
# GET THE INITIAL ORBITS OF THE SYMM GROUP
cut_orbits = []
hash_table = {}
node_table = {}
saucy_out_file = open("./saucy_gens.txt", "w")
saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                            stderr = saucy_out_file, shell = False)
saucy_return, saucy_err = saucy_out.communicate()
sage_gens = read_saucy_gens("./saucy_gens.txt", o_mdl_cols, o_mdl_rows)
sage_perm_group = PermutationGroup(sage_gens)
print("\nfirst iteration\n")
sage_orbits = copy.deepcopy(sage_perm_group.orbits())
orbit_part = orbital_partition(o_mdl, sage_orbits)
agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
solve_cplex(agg_mdl)
new_orbits, cut_hash_table, cut_node_hash = generate_cuts(o_mdl, agg_mdl, sage_perm_group, sage_perm_group, orbit_part)
cut_orbits += new_orbits
hash_table.update(cut_hash_table)
node_table.update(cut_node_hash)
solve_cplex(agg_mdl)
input()

print("\nsecond iteration\n")
stab_group = sage_perm_group.stabilizer(0)
sage_orbits = copy.deepcopy(stab_group.orbits())
cut_orbits = split_cut_orbits(cut_orbits, node_table, hash_table, stab_group)
sage_orbits += cut_orbits
print(stab_group.orbits())
orbit_part = orbital_partition(o_mdl, sage_orbits)
agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
solve_cplex(agg_mdl)
agg_mdl.solution.write("cplex_agg_solution_before.txt")
new_orbits, cut_hash_table, cut_node_hash = generate_cuts(o_mdl, agg_mdl, stab_group, sage_perm_group, orbit_part)
cut_orbits += new_orbits
hash_table.update(cut_hash_table)
node_table.update(cut_node_hash)
solve_cplex(agg_mdl)
agg_mdl.solution.write("cplex_agg_solution_after.txt")
input()

print("\nthird iteration\n")
stab_group = stab_group.stabilizer(1)
sage_orbits = copy.deepcopy(stab_group.orbits())
cut_orbits = split_cut_orbits(cut_orbits, node_table, hash_table, stab_group)
sage_orbits += cut_orbits
print(stab_group.orbits())
print(sage_perm_group.orbits())
orbit_part = orbital_partition(o_mdl, sage_orbits)
agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
solve_cplex(agg_mdl)
agg_mdl.solution.write("cplex_agg_solution_before.txt")
new_orbits, cut_hash_table, cut_node_hash = generate_cuts(o_mdl, agg_mdl, stab_group, sage_perm_group, orbit_part)
cut_orbits += new_orbits
hash_table.update(cut_hash_table)
node_table.update(cut_node_hash)
solve_cplex(agg_mdl)
agg_mdl.solution.write("cplex_agg_solution_after.txt")
s1 = sorted(hash_table.keys())
s2 = sorted(node_table.values())
freq = {val : 0 for val in s1}
for s in s2:
    freq[s] += 1
for s, v in freq.items():
    if v > 1:
        print(s, v)

# print("\nfourth iteration\n")
# stab_group = stab_group.stabilizer(3)
# sage_orbits = copy.deepcopy(stab_group.orbits())
# cut_orbits = split_cut_orbits(cut_orbits, node_table, hash_table, stab_group)
# sage_orbits += cut_orbits
# for i in range(18, 463):
#     if i not in hash_table.values():
#         print(i)

# orbit_part = orbital_partition(o_mdl, sage_orbits)
# agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
# solve_cplex(agg_mdl)
# agg_mdl.solution.write("cplex_agg_solution_before.txt")
# new_orbits, cut_hash_table, cut_node_hash = generate_cuts(o_mdl, agg_mdl, stab_group, sage_perm_group, orbit_part)
# cut_orbits += new_orbits
# hash_table.update(cut_hash_table)
# node_table.update(cut_node_hash)
# solve_cplex(agg_mdl)
# agg_mdl.solution.write("cplex_agg_solution_after.txt")


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS1
Selected bound      name:  BND1

first iteration

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 1 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
[1.0, 0.2] | 1.8
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1

Iteration log . . .
Iteration:     1   Dual objective     =             2.000000


second iteration

[[0], [1, 2, 6, 3], [4, 5, 7, 8], [9], [10, 11, 15, 12], [13, 14, 16, 17]]
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =             2.000000
[0.0, 1.0, 0

[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, -0.5, 0.0, 0.0, 0.125, 0.0, 0.0] | 0.5
[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, -0.125, 0.0, 0.0] | 0.5
[-1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -0.5, 0.0, -1.0, -0.5, 0.0, 0.0, 0.375, 0.0, 0.0] | 0.5
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.5, 0.0, 1.0, 0.5, 0.0, 0.0, -0.125, 0.0, 0.0] | 0.5
[-2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.5, 0.0, 0.0] | 0.0
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.75, 0.0, 0.0] | 1.0
[2.0, 0.0, -2.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0] | 0.0
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.75, -1.0, 0.0] | 0.0
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0] | 0.0
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1

Iteration log . . .
Iteration:     1   Dual objective     =             2.000000
2.,1.,1.,2.,1.,1.,2.,1.,1. 2


In [224]:
m = gp.read("cut_added_agg_model_cplex.lp")
m.optimize()

Read LP format model from file cut_added_agg_model_cplex.lp
Reading time = 0.01 seconds
obj1: 6 rows, 6 columns, 24 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 6 rows, 6 columns and 24 nonzeros
Model fingerprint: 0x4981912d
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 2e+00]
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.000000e+00   0.000000e+00      0s
       6    1.8000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.800000000e+00


In [225]:
m.write("gur_sol.sol")